Read the wiki page containing postal codes through Beaautiful soup library

In [33]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
#html_page='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_page=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
#page = urlopen(html_page)
#html_doc="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
soup = BeautifulSoup(html_page, 'lxml')

#print(soup.prettify())

In [34]:
#some defs
def lens(listoflists):
  return [len(x) for x in listoflists]

grab the relevant section from wiki page

In [35]:
relevant_soup=soup.find('table',class_='wikitable sortable')
print(relevant_soup.prettify())

<table class="wikitable sortable">
 <tbody>
  <tr>
   <th>
    Postcode
   </th>
   <th>
    Borough
   </th>
   <th>
    Neighbourhood
   </th>
  </tr>
  <tr>
   <td>
    M1A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M2A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M3A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Parkwoods" title="Parkwoods">
     Parkwoods
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M4A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Victoria_Village" title="Victoria Village">
     Victoria Village
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M5A
   </td>
   <td>
    <a href="/wiki/Downtown_Toronto" title="Downtown Toronto">
     Downtown Toronto
    </a>
   </td>
   <td>
    <a href="

Get he column headings of the dataframe

In [36]:
relevant_column_headings=relevant_soup.find('tr')
#i=0
columns=list()
#columns.append('great')
for column_titles in relevant_column_headings.find_all('th'):
    #columns[i]=column_titles.text
    columns.append(column_titles.text)
    print(column_titles.text)
    #i=i+1
columns = list(map(lambda s: s.strip(), columns))
print(columns)

Postcode
Borough
Neighbourhood

['Postcode', 'Borough', 'Neighbourhood']


Now get the entire postcode,borough and neighbourhood in a table..include the column headings as of now

In [37]:
Grab_table_contents=soup.find('tbody')
#print(Grab_table_contents)
Grab_table=[[]]
Grab_food=[]
for relevant_column_headings in Grab_table_contents.find_all('tr'):
    Grab_food=[]
#columns.append('great')
    for column_titles in relevant_column_headings.find_all('td'):
        #columns[i]=column_titles.text
        Grab_food.append(column_titles.text)
        #print(Grab_food)
    Grab_food = list(map(lambda s: s.strip(), Grab_food))
    Grab_table.append(Grab_food)
    
#print(Grab_table)
import numpy as np
Grab_table=np.asarray(Grab_table)
Grab_table = [x for x in Grab_table if x]
#print(Grab_table[0:5])
import pandas as pd
#Postal_df=pd.DataFrame(Grab_table=Grab_table[1:,1:],index=Grab_table[1:,0])
#Postal_df = pd.DataFrame({'Column1':Grab_table[:,0],'Column2':Grab_table[:,1],'Column3':Grab_table[:,2]})

asg=0
sg=0
for x in Grab_table:
    sg=len(x)
  
    if sg==3:
        sg=sg
    else:
         asg=asg+1
            
            
#print("abnomal  is:",asg)
        
    

#print(Postal_df)

Create dataframee from the numy arrays we have created.
clean up not assigned values and merge neighbourhood if duplicate of same postcode and borough

In [38]:
df_post=pd.DataFrame(Grab_table, columns=columns)

#group by postcode
#df_post_filter=df_post.groupby("postcode")
#f_post_filter.head()
df_post['Neighbourhood'] = df_post[['Postcode','Borough','Neighbourhood']].groupby(['Postcode','Borough'])['Neighbourhood'].transform(lambda x: ','.join(x))


    

df_post.head(20)
df_post.shape

(289, 3)

In [39]:
print(df_post.shape)
df_post = df_post.drop(df_post[df_post.Borough == 'Not assigned'].index)

r,c=df_post.shape
i=0

for i in range(r):
    if (df_post.iloc[i,2] == 'Not assigned'):
        df_post.iloc[i,2] = df_post.iloc[i,1]
  #      df_post.drop([df_post.index[i]])
#df_post = df_post.drop(df_post[df_post.Borough == 'Not assigned'].index)
df_post.head()
#print(df_post.shape)

(289, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront,Regent Park"
5,M5A,Downtown Toronto,"Harbourfront,Regent Park"
6,M6A,North York,"Lawrence Heights,Lawrence Manor"


Get the geo coordinates the address through geocoder library.
Geocoder not working for some reason
Comment geocoder code and fetch from cv file

In [40]:
#!conda install -c conda-forge geocoder

#import geocoder # import geocoder

# initialize your variable to None
#lat_lng_coords = None
#postal_code='M5G'

# loop until you get the coordinates
#while(lat_lng_coords is None):
 # g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  #lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]
#print("latitude is",latitude)
#print("longitude is",longitde)

In [41]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#now map lat,long to df_datapost

In [52]:
df_post["Latitude"]=""
df_post["Longitude"]=""
rows,cols= df_post.shape
print(df_post.shape)
index=0
colid=df_post.columns.get_loc("Postcode")
colid_la=df_post.columns.get_loc("Latitude")
colid_long=df_post.columns.get_loc("Longitude")
cid_la=df_data_1.columns.get_loc("Latitude")
cid_long=df_data_1.columns.get_loc("Longitude")


for index in range(rows):
    pc=df_post.iloc[index,colid]
    rid=df_data_1[df_data_1['Postal Code'] == pc].index.tolist()
    lengthofrid=len(rid)
    if (lengthofrid == 1):
        df_post.iloc[index,colid_la] = df_data_1.iloc[rid[0],cid_la]
        df_post.iloc[index,colid_long] = df_data_1.iloc[rid[0],cid_long]
    else:
        print("duplicate or not found",rid,pc,lengthofrid)

df_post.head()


(212, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.7533,-79.3297
3,M4A,North York,Victoria Village,43.7259,-79.3156
4,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.6543,-79.3606
5,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.6543,-79.3606
6,M6A,North York,"Lawrence Heights,Lawrence Manor",43.7185,-79.4648
